In [7]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD= os.getenv('POSTGRES_PASSWORD')

In [30]:
bills=pd.read_csv('Data/bills.csv')  
committee_members=pd.read_csv('Data/committee_members.csv') 
committess=pd.read_csv('Data/committees.csv')  
congress_members=pd.read_csv('Data/congress_members.csv') 
leadership=pd.read_csv('Data/leadership.csv')  
od_contributors=pd.read_csv('Data/os_contributions.csv') 
os_legislators=pd.read_csv('Data/os_legislators.csv') 
terms=pd.read_csv('Data/terms.csv')  
votingaffinity=pd.read_csv('Data/votingaffinity.csv')

# Bills

In [31]:
bills.columns= [x.lower() for x in bills.columns]
bills.columns= [x.replace('.','_') for x in bills.columns]
bills.columns

Index(['congress', 'introduceddate', 'number', 'title', 'type', 'url',
       'latestaction_actiondate', 'latestaction_text', 'policyarea_name',
       'amendmentnumber', 'latestaction', 'latestaction_actiontime'],
      dtype='object')

In [32]:
bills.head(3).T

,0,1,2
congress,118,118,118
introduceddate,2023-10-04,2023-09-29,2023-09-27
number,405.0,2998.0,373.0
title,A resolution expressing support for the design...,Land-Grant Research Equity and Accountability Act,A resolution designating the week of September...
type,SRES,S,SRES
url,https://api.congress.gov/v3/bill/118/sres/405?...,https://api.congress.gov/v3/bill/118/s/2998?fo...,https://api.congress.gov/v3/bill/118/sres/373?...
latestaction_actiondate,2023-10-04,2023-09-29,2023-09-27
latestaction_text,"Submitted in the Senate, considered, and agree...",Read twice and referred to the Committee on Ag...,"Submitted in the Senate, considered, and agree..."
policyarea_name,Armed Forces and National Security,Agriculture and Food,NaN
amendmentnumber,NaN,NaN,NaN


In [33]:
congress_members.columns= [x.lower() for x in congress_members.columns]
congress_members.columns= [x.replace('.','_') for x in congress_members.columns]

congress_members.columns



Index(['bioguideid', 'birthyear', 'currentmember', 'directordername',
       'firstname', 'honorificname', 'invertedordername', 'lastname',
       'officialwebsiteurl', 'partyhistory', 'state', 'terms', 'updatedate',
       'addressinformation_city', 'addressinformation_district',
       'addressinformation_officeaddress', 'addressinformation_phonenumber',
       'addressinformation_zipcode', 'cosponsoredlegislation_count',
       'cosponsoredlegislation_url', 'depiction_attribution',
       'depiction_imageurl', 'sponsoredlegislation_count',
       'sponsoredlegislation_url', 'middlename', 'suffixname', 'nickname',
       'leadership', 'district'],
      dtype='object')

# congress_members

In [34]:
congress_members=congress_members.drop(['terms', 'leadership', 'partyhistory'], axis=1)

In [15]:
congress_members.head(3).T

,0,1,2
bioguideid,B000944,C000127,C000141
birthyear,1952,1958,1943
currentmember,True,True,True
directordername,Sherrod Brown,Maria Cantwell,Benjamin L. Cardin
firstname,Sherrod,Maria,Ben
honorificname,Mr.,Ms.,Mr.
invertedordername,"Brown, Sherrod","Cantwell, Maria","Cardin, Benjamin L."
lastname,Brown,Cantwell,Cardin
officialwebsiteurl,https://www.brown.senate.gov/,https://www.cantwell.senate.gov,https://www.cardin.senate.gov/
partyhistory,"[{'partyAbbreviation': 'D', 'partyName': 'Demo...","[{'partyAbbreviation': 'D', 'partyName': 'Demo...","[{'partyAbbreviation': 'D', 'partyName': 'Demo..."


# Terms

In [16]:
terms.columns= [x.lower() for x in terms.columns]
terms.columns= [x.replace('.','_') for x in terms.columns]

terms.columns

Index(['chamber', 'congress', 'endyear', 'membertype', 'startyear',
       'statecode', 'statename', 'bioguideid', 'district'],
      dtype='object')

In [18]:
terms.head(3).T

,0,1,2
chamber,Senate,Senate,Senate
congress,107,108,109
endyear,2003.0,2005.0,2007.0
membertype,Senator,Senator,Senator
startyear,2002,2003,2005
statecode,TX,TX,TX
statename,Texas,Texas,Texas
bioguideid,C001056,C001056,C001056
district,NaN,NaN,NaN


In [17]:
leadership.columns= [x.lower() for x in leadership.columns]
leadership.columns= [x.replace('.','_') for x in leadership.columns]

leadership.columns

Index(['congress', 'type', 'bioguideid', 'current'], dtype='object')

In [19]:
leadership.head(3).T

,0,1,2
congress,113,114,115
type,Assistant Democratic Leader,Assistant Majority Leader,Majority Whip
bioguideid,C001056,C001056,C001056
current,NaN,NaN,NaN


# os_legislators

In [27]:
os_legislators.columns= [x.lower() for x in os_legislators.columns]
os_legislators.columns= [x.replace('.','_') for x in os_legislators.columns]

os_legislators.columns

Index(['cid', 'firstl', 'lastnam', 'party', 'offic', 'gend', 'first_elected',
       'xit_cod', 'commen', 'phon', 'fax', 'w', 'webform', 'congress_offic',
       'oguide_id', 'votesmart_id', 'feccandid', 'witter_id', 'youtube_url',
       'facebook_id', 'hd'],
      dtype='object')

In [28]:
os_legislators.head(3).T

,0,1,2
cid,N00050780,N00035774,N00026050
firstl,Mary Peltola,Dan Sullivan,Lisa Murkowski
lastnam,Peltola,Sullivan,Murkowski
party,D,R,R
offic,AK01,AKS1,AKS2
gend,F,M,F
first_elected,2022,2014,2002
xit_cod,0,0,0
commen,NaN,NaN,NaN
phon,NaN,202-224-3004,202-224-6665
